<a href="https://colab.research.google.com/github/OlegPozovnoy/MailRuMLHomeworks/blob/master/HW3/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/datasets/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /datasets/


In [3]:
!ls "/datasets/My Drive/datasets/"

players.pkl  results.pkl  tournaments.pkl


In [0]:
import pickle
import pandas as pd

In [0]:
tournaments = pd.read_pickle("/datasets/My Drive/datasets/tournaments.pkl")
#tournaments = pd.read_pickle("./tournaments.pkl")

tournaments_cleaned = [(v['id'], v['name'], v['dateStart'][0:4], v['type']['name'], v['questionQty']) for k,v in tournaments.items() if v['questionQty'] != None]
del tournaments

Создадим трейн и тест набор турниров.

In [6]:
train_id = [ v[0] for v in tournaments_cleaned if int(v[2])== 2019]
test_id = [v[0] for v in tournaments_cleaned if int(v[2])== 2020]

print(len(train_id))
print(len(test_id))

687
418


In [0]:
results = pd.read_pickle("/datasets/My Drive/datasets/results.pkl")
#results = pd.read_pickle("./results.pkl")
results_cleaned = {k: v for k,v in results.items() if k in train_id or k in test_id}

results_cleaned2 = {}
for k,v in results_cleaned.items():
  results_cleaned2[k] = []
  for s in v:
    if s.get('mask') != None:
      results_cleaned2[k].append((s.get('mask'),s.get('questionsTotal'), [k.get('player').get('id') for k in s.get('teamMembers')]))

del results
del results_cleaned

In [8]:
train_id = [id for id in train_id if id in results_cleaned2.keys() and len(results_cleaned2[id]) > 0]
test_id = [id for id in test_id if id in results_cleaned2.keys() and len(results_cleaned2[id]) > 0]

print(len(train_id))
print(len(test_id))

675
173


In [9]:
import numpy as np
from sklearn.linear_model import LogisticRegression
import tqdm
import gc

# creating all questions and players set
all_words = []
for k,v in results_cleaned2.items():
  for team in v: 
    for p in range(len(team[0])):
        all_words.append(str(k)+'_'+str(p))
    for m in team[2]:
        all_words.append(m)

all_words = set(all_words)
print(len(all_words))

# creating storage for competetion results
all_results = {}
for s in all_words:
  all_results[s] = [0,0] # total rating and num of competetions


logistic = LogisticRegression(fit_intercept = False)

def process_tournament(id):
  task3 = []
  words = []
   
  for n in range(len(results_cleaned2[id])): # command id
    for p in range(len(results_cleaned2[id][n][0])): #answers
        quest = str(id)+'_'+str(p)
        team = n
        task3.append((quest, team,results_cleaned2[id][n][0][p]) )
        words.append(quest)
        words.append(team)

  if len(words)==0:
    return 
  
  words = set(words) # encoding questions and teams no

  idx_to_word = dict(enumerate(words))
  word_to_idx = dict([(y,x) for x,y in idx_to_word.items()])
  #preparing matrix
  X = np.zeros((len(task3),len(words)))
  Y = np.zeros(len(task3))
  for i in range(len(task3)):
    X[i,word_to_idx[task3[i][0]]] = 1
    X[i,word_to_idx[task3[i][1]]] = 1
    Y[i] = 1 if str(task3[i][2]) == '1' else 0

  if len(set(Y)) <= 1:
    print ('<2 categories , skipping')
    return

  #print(X.shape)
  logistic.fit(X, Y)

  for s in range(len(logistic.coef_[0])):
    if isinstance(idx_to_word[s], int): ## teanm no
        #print('team '+ str(idx_to_word[s]))
        for player in results_cleaned2[id][idx_to_word[s]][2]:
            #print(player)
            all_results[player][1] +=1
            all_results[player][0] +=logistic.coef_[0][s]
    else: #question no
        #print('question '+ str(idx_to_word[s]))
        all_results[idx_to_word[s]][1] +=1
        all_results[idx_to_word[s]][0] +=logistic.coef_[0][s]

#process_tournament(1526)


104707


Пройдемся по всем турнирам и посчитаем рейтинг всех игроков и вопросов в заданном турнире, за неимением лучшего, усредним по каждому игроку рейтинг в турнире

In [10]:
import tqdm

for tourn_id in tqdm.tqdm(train_id):
  process_tournament(tourn_id)

 99%|█████████▉| 671/675 [01:47<00:00, 30.26it/s]

<2 categories , skipping


100%|██████████| 675/675 [02:34<00:00,  4.37it/s]


In [11]:
import statistics as stat

def is_player(t):
  return isinstance(t, int)

all_ranking = {}

for k,v in all_results.items():
  if v[1] != 0:
    all_ranking[k] = v[0]/v[1] 

blank_rank = stat.mean ([ v for k,v in all_ranking.items() if is_player(k)])
print(blank_rank)


-0.5379797641338596


In [0]:
def get_team_ranking(players):
  global all_ranking
  result = []
  #print('Team:',players)
  if (len(players)==0):
    return blank_rank
  for player in players:
    if (all_results[player][1]>0): # num of competetions
      #print(player, ' rating is ', all_ranking[player])
      result.append(all_ranking[player])
    else:
      result.append(blank_rank)
  return stat.mean(result)


In [13]:
from scipy import stats

spearman = []
pearson = []

test_id = [x for x in test_id if len(results_cleaned2[x])>1] ## фильтруем турниры с 1 командой

for tourn_id in test_id:
  #print('Tournament:', tourn_id)
  rating = []
  ranking = []
  i=1
  for team in results_cleaned2[tourn_id]:
    rating.append(get_team_ranking(team[2]))
    ranking.append(-i)
    i+=1
  #print ('Spearman: ',stats.spearmanr(rating, ranking)[0])
  #print ('Pearson: ',stats.pearsonr(rating,ranking)[0])
  # ставим минус т.к. у нас места возрастают а сила убывает
  spearman.append(stats.spearmanr(rating, ranking)[0])
  pearson.append(stats.pearsonr(rating,ranking)[0])

print(spearman)
print(pearson)
print(stat.mean(spearman))
print(stat.mean(pearson))


[0.7302837415039378, 0.7769638794729264, 0.7857266542980827, 0.7571551572305485, 0.5878069812176181, 0.721943417706411, 0.6338391968235293, 0.7020631281960853, 0.5958556798082136, 0.4168306855170206, 0.5182010463585892, 0.8440030750950487, 0.6928426044300088, 0.6432143638468918, 0.6397192764003768, 0.5945538602214889, 0.8308823529411766, 0.7972044866550946, 0.8032883294648357, 0.7004561824729891, 0.821067714289558, 0.8381972265023113, 0.7826655282623971, 0.8164776477647764, 0.6346326739079873, 0.7690484126021944, 0.755033462365203, 0.617391304347826, 0.8361712453244572, 0.787268817204301, 0.8869021782814887, 0.8590717678970162, 0.764092326055874, 0.7668308702791461, 0.882706766917293, 0.8022405775523899, 0.748051948051948, 0.7898180910906758, 0.7693191013030302, 0.7615320790320791, 0.812891986062718, 0.7277026368207713, 0.7622377622377624, 0.9009304871373838, 0.7033580067453409, 0.8251082251082253, 0.8128977114315507, 0.8183650615901457, 0.7867509848641924, 0.8587219541924115, 0.818100

Теперь будем работать в рамках модели, когда участники команды отвечают на вопрос независимо и если кто то ответил - команда отвечает правильно, если никто - не отвечает. Будем попеременно улучшать оценки для сложностей вопросов и индивидуальных показателей всех участников

In [0]:
from scipy.special import expit
import math

def sigmoid(x):
  return expit(x)

def sigm_reverse(x):
  if x<0.001:
    return -10
  elif x>0.999:
    return 10
  result = -math.log(1/x-1)
  if (result >6):
    return 6
  elif(result < -6):
    return -6
  else:
    return result

def adjust_individual(players, team_X, Y):
  global all_ranking
  #print('Adjust individual rank by team rank: ', players, team_X, Y)
  prob_real = 1-sigmoid(team_X + Y)
  if len(players) ==0:
    return
  #print('Prob_real:',prob_real)
  prob_adjusted = 1
  probs = []
  player_ratings = []
  for player in players:
    player_rating = all_ranking[player]
    player_ratings.append(player_rating)
    player_prob = (1 - sigmoid(player_rating + Y))
    probs.append(player_prob)
    prob_adjusted *= (1 - sigmoid(player_rating + Y))
  #print('Prob_adjusted', prob_adjusted)
  multiplier = math.pow(prob_real/prob_adjusted, 1.0/len(players))
  #print(multiplier)
  probs = [x*multiplier for x in probs]
  #print(probs)
  #print([sigm_reverse(1-x) - Y for x in probs])
  result = [sigm_reverse(1-x) - Y for x in probs] 
  #print ('Before adjustment: ', player_ratings)
  #print ('After adjustment: ', result)  
  return result

def compose_team(players, Y):
  global blank_rank
  global all_ranking
  #print('Get team rating by individual: ', players, team_X)
  prob_adjusted = 1
  players_ranks = []
  for player in players:
    if player in all_ranking:
      player_rating = all_ranking[player]
    else:
      player_rating = blank_rank
    players_ranks.append(player_rating)
    #print('player:',player,'rating:',player_rating)
    prob_adjusted *= (1 - sigmoid(player_rating + Y))
  #print('prob_by_member:', prob_adjusted)
  #print('prob_by_condition', 1-sigmoid(team_X+Y))
  #print('Players rank:',player_ranks)
  result = sigm_reverse(1-prob_adjusted) - Y 
  #print('Team rank: ', result)
  return result

def get_team_rank(players):
  global all_ranking
  if len(players)==0:
    return sigm_reverse(1 - (1 - sigmoid(blank_rank)**5))
  prob = 1
  players_ranks = []
  for player in players:
    if player in all_ranking:
      player_rating = all_ranking[player]
    else:
      player_rating = blank_rank
    players_ranks.append(player_rating)
    player_prob = (1 - sigmoid(player_rating))
    prob *= player_prob
  #print ('Players ranks: ',player_ranks)
  result = sigm_reverse(1-prob)
  #print('Team rank:') 
  return result

In [0]:
# creating storage for competetion results

logistic = LogisticRegression(fit_intercept = False)

def record_item(item, value):
  global all_results_upd
  all_results_upd[item][1] += 1
  all_results_upd[item][0] += value

def upd_EM(id, update_questions):
  global all_ranking
  task3 = []
  words = []
   
  for n in range(len(results_cleaned2[id])): # command id
    for p in range(len(results_cleaned2[id][n][0])): #answers
        quest = str(id)+'_'+str(p)
        team = n
        task3.append((quest, team,results_cleaned2[id][n][0][p]))
        words.append(quest)
        words.append(team)

  if len(words)==0:
    return 
  
  words = set(words) # encoding questions and teams no

  idx_to_word = dict(enumerate(words))
  word_to_idx = dict([(y,x) for x,y in idx_to_word.items()])
  #preparing matrix
  X = np.zeros((len(task3),len(words)))
  Y = np.zeros(len(task3))
  for i in range(len(task3)):
    X[i,word_to_idx[task3[i][0]]] = 1
    X[i,word_to_idx[task3[i][1]]] = 1
    Y[i] = 1 if str(task3[i][2]) == '1' else 0

  if len(set(Y)) <= 1:
    print ('<2 categories , skipping')
    return

  logistic.fit(X, Y)

  for i in range(len(task3)):
    X[i,word_to_idx[task3[i][0]]] = 1
    X[i,word_to_idx[task3[i][1]]] = 1
    Y[i] = 1 if str(task3[i][2]) == '1' else 0


  for i in range(len(task3)):
    question = word_to_idx[task3[i][0]]
    team = word_to_idx[task3[i][1]]
    k_quest = logistic.coef_[0][question]
    k_team = logistic.coef_[0][team]
    players_list = results_cleaned2[id][task3[i][1]][2]
    if (update_questions):
      K = compose_team(players_list, all_ranking[task3[i][0]])
      record_item(task3[i][0], k_quest + k_team - K)
    else: # team members
      #print(players_list, k_team, task3[i][0],all_ranking[task3[i][0]])
      ranking_split = adjust_individual(players_list, k_team,all_ranking[task3[i][0]])
      for j in range(len(players_list)):
        record_item(players_list[j], ranking_split[j])

In [16]:
import tqdm

all_results_upd = {}
for s in all_words:
  all_results_upd[s] = [0,0] # total rating and num of competetions

def record_new_ratings():
  global all_ranking
  global blank_rank
  global all_results_upd
  for k,v in all_results_upd.items():
    if v[1] != 0:
      all_ranking[k] = v[0]/v[1] 
  print('blank_rank before',blank_rank)
  blank_rank = stat.mean ([ v for k,v in all_ranking.items() if is_player(k)])
  print('blank_rank after',blank_rank)

  all_results_upd = {}
  for s in all_words:
    all_results_upd[s] = [0,0] # total rating and num of competetions


def calc_corellations():
  spearman = []
  pearson = []

  for tourn_id in test_id:
    #print('Tournament:', tourn_id)
    rating = []
    ranking = []
    i=1
    for team in results_cleaned2[tourn_id]:
      rating.append(get_team_rank(team[2]))
      ranking.append(-i)
      i+=1
    #print ('Spearman: ',stats.spearmanr(rating, ranking)[0])
    #print ('Pearson: ',stats.pearsonr(rating,ranking)[0])
    # ставим минус т.к. у нас места возрастают а сила убывает
    spearman.append(stats.spearmanr(rating, ranking)[0])
    pearson.append(stats.pearsonr(rating,ranking)[0])
  
  print('Spearman: ',stat.mean(spearman))
  print('Pearson: ',stat.mean(pearson))  


def EM_cycle():  
  print('Updating individual ranking')
  for tourn_id in tqdm.tqdm(train_id):
    upd_EM(tourn_id, update_questions = False)

  record_new_ratings()
  print('Updating questions')
  for tourn_id in tqdm.tqdm(train_id):
    upd_EM(tourn_id, update_questions = True)

  calc_corellations()

EM_cycle()

  0%|          | 0/675 [00:00<?, ?it/s]

Updating individual ranking


 98%|█████████▊| 664/675 [04:39<00:00, 12.04it/s]

<2 categories , skipping


  0%|          | 0/675 [00:00<?, ?it/s]

blank_rank before -0.5379797641338596
blank_rank after -3.5750687979852587
Updating questions


 99%|█████████▊| 666/675 [03:11<00:00, 15.69it/s]

<2 categories , skipping


100%|██████████| 675/675 [04:08<00:00,  2.72it/s]


NameError: ignored

Spearman:  0.6457651887789548
Pearson:  0.6227593483707757


In [21]:
for i in range(10):
  print('итерация ',i)
  EM_cycle()

  0%|          | 0/675 [00:00<?, ?it/s]

итерация  0
Updating individual ranking


 98%|█████████▊| 664/675 [04:40<00:00, 12.04it/s]

<2 categories , skipping


  0%|          | 0/675 [00:00<?, ?it/s]

...

Updating questions


 98%|█████████▊| 663/675 [03:08<00:00, 16.68it/s]

<2 categories , skipping


100%|██████████| 675/675 [04:07<00:00,  2.73it/s]


Spearman:  0.681531978724274
Pearson:  0.6379497417482434
